### Exercise:
- Scene을 가장 정확하고 자연스럽게 복원할 수 있는 최적의 세팅을 세 가지 관점(Model 종류, Model 크기, Optimization)의 관점에서 찾고, 이를 Report(양식자유)로 작성하여 드라이브에 제출해주세요.
    - Model 종류:
        - NeRFacto
        - SplatFacto
        - Instant-NGP
    - Model 크기:
        - hidden_dim of MLP
        - num_levels of Hash Map
    - Optimization
        - learning_rate
- 본 튜토리얼에서 3가지의 다른 모델(NeRFacto, SplatFacto, Instant-NGP)을 차례대로 돌려보시게 됩니다. 각각의 모델에서 최적의 세팅을 찾아서 보고서에 작성해주세요.
- 가장 마지막에 3가지 모델의 성능의 비교해서 정리해주세요.
- 학습 과정 및 평가 결과를 log하기 위해서 wandb를 사용합니다(https://wandb.ai). Wandb 회원 가입 및 API_KEY 발급을 진행해주세요

In [ ]:
import os
from pathlib import Path

NERFSTUDIO_PORT = os.environ["NERFSTUDIO_PORT"]

In [ ]:
try:
    import getpass
    api_key = getpass.getpass("Enter your Wandb API key: ")
except Exception:
    print("Note: Your API key will be visible when typing.")
    api_key = input("Enter your Wandb API key: ")

# Set the API key as an environment variable
os.environ["WANDB_API_KEY"] = api_key

# Login to wandb
try:
    wandb.login(key=api_key, relogin=True)
    print("✅ Successfully logged in to Wandb!")
    print(f"✅ You are logged in as: {wandb.api.default_entity}")
except Exception as e:
    print(f"❌ Login failed: {e}")
    print("Please check your API key and try again.")

In [ ]:
scene_names = []

for p in sorted(Path("/data/nerfstudio").glob("*")):
    if p.is_dir():
        scene_names.append(p.name)

print(scene_names)

In [ ]:
import getpass

scene_names = []
for p in sorted(Path("/data/nerfstudio").glob("*")):
    if p.is_dir():
        scene_names.append(p.name)

print("Existing scenes:")
print(scene_names)
print("\n")
        
SCENE = getpass.getpass("Select a scene: ")

while SCENE not in scene_names:
    print("Existing scenes:")
    print(scene_names)
    SCENE = input("Select a scene among the existing scenes: ")

## 1. Nerfacto

In [ ]:
!ns-train \
    nerfacto \
    --output-dir "results/nerfstudio/nerfacto" \
    --experiment-name "nerfacto_base" \
    --vis viewer+wandb  \
    --viewer.websocket-port "{NERFSTUDIO_PORT}" \
    nerfstudio-data \
    --data /data/nerfstudio/"{SCENE}" \
    --downscale-factor 4 

### Customize the Nerfacto model
- Refer to `!ns-train nerfacto --help` and [nerfacto.py](https://github.com/nerfstudio-project/nerfstudio/blob/main/nerfstudio/models/nerfacto.py)
- Recommended options to ablate:
    - model
        - pipeline.model.hidden-dim
        - pipeline.model.hidden-dim-color
        - pipeline.model.hidden-dim-transient
        - pipeline.model.num-levels
        - pipeline.model.base-res
        - pipeline.model.max-res
        - pipeline.model.log2-hashmap-size
        - pipeline.model.feature-per-level
    - optimizer
        - optimizers.fields.optimizer.lr
        - optimizers.fields.optimizer.eps
        - optimizers.fields.optimizer.max-norm
        - optimizers.fields.optimizer.weight-decay

In [ ]:
!ns-train \
    nerfacto \
    --pipeline.model.hidden-dim 128 \
    --pipeline.model.hidden-dim-color 128 \
    --pipeline.model.hidden-dim-transient 128 \
    --output-dir "results/nerfstudio/nerfacto" \
    --experiment-name "nerfacto_exp_00" \
    --vis viewer+wandb  \
    --viewer.websocket-port "{NERFSTUDIO_PORT}" \
    nerfstudio-data \
    --data /data/nerfstudio/"{SCENE}" \
    --downscale-factor 4

## 2.Instant-NGP

In [ ]:
!ns-train \
    instant-ngp \
    --output-dir "results/nerfstudio/instant-ngp" \
    --experiment-name "instant-ngp" \
    --vis viewer+wandb  \
    --viewer.websocket-port "{NERFSTUDIO_PORT}" \
    nerfstudio-data \
    --data /data/nerfstudio/"{SCENE}" \
    --downscale-factor 4

### Customize the Instant-NGP model
- Refer to `!ns-train instant-ngp --help` and [instant_ngp.py](https://github.com/nerfstudio-project/nerfstudio/blob/main/nerfstudio/models/instant_ngp.py)
- Recommended options to ablate:
    - model
        - pipeline.model.grid-resolution
        - pipeline.model.hidden-dim-color
        - pipeline.model.hidden-dim-transient
        - pipeline.model.num-levels
        - pipeline.model.base-res
        - pipeline.model.max-res
        - pipeline.model.log2-hashmap-size
        - pipeline.model.feature-per-level
    - optimizer
        - optimizers.fields.optimizer.lr
        - optimizers.fields.optimizer.eps
        - optimizers.fields.optimizer.max-norm
        - optimizers.fields.optimizer.weight-decay

## 3. Splatfacto

In [ ]:
!echo "y" | ns-train \
    splatfacto \
    --output-dir "results/nerfstudio/splatfacto" \
    --experiment-name "splatfacto" \
    --vis viewer+wandb  \
    --viewer.websocket-port "{NERFSTUDIO_PORT}" \
    nerfstudio-data \
    --data /data/nerfstudio/"{SCENE}" \
    --downscale-factor 4

### Customize the Splatfacto model
- Refer to `!ns-train splatfacto --help` and [splatfacto.py](https://github.com/nerfstudio-project/nerfstudio/blob/50e0e3c70c775e89333256213363badbf074f29d/nerfstudio/models/splatfacto.py#L87)
- Recommended options to ablate:
    - model
        - pipeline.model.warmup-length
        - pipeline.model.refine-every
        - pipeline.model.cull-alpha-thresh
        - pipeline.model.cull-scale-thresh
        - pipeline.model.reset-alpha-every
        - pipeline.model.densify-grad-thresh
        - pipeline.model.densify-size-thresh
        - pipeline.model.n-split-samples
        - pipeline.model.ssim-lambda
 
    - gs means optimizer
        - optimizers.means.optimizer.lr
        - optimizers.means.optimizer.eps
        - optimizers.means.optimizer.max-norm
        - optimizers.means.optimizer.weight-decay

## 4. Discover more scenes
https://docs.nerf.studio/quickstart/existing_dataset.html